In [46]:
#首先将用到的包进行导入
import pandas as pd
import numpy as np
import jieba
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection 
from sklearn import preprocessing

In [47]:
#将数据进行读取
data=pd.read_csv('./new_reviews_score_update.csv',index_col=0)
#现在是划分数据集
#random_state 取值，这是为了在不同环境中，保证随机数取值一致，以便验证模型的实际效果。
train_x,test_x,train_y,test_y=model_selection.train_test_split(data.评论.values.astype('U'),data.评分.values,test_size=0.2,random_state=1)
 
#划分完毕，查看数据形状
print(train_x.shape,test_x.shape)
#train_x 训练集数据 test_x 测试集数据  train_y训练集的标签 test_y 测试集的标签
t_y=test_y

(7703,) (1926,)


In [48]:

def build_dict():
    #构建词典
    all_text=data['评论'].values.astype('U')
    crops_dict={}
    for sentence in all_text:
        list=sentence.split(' ')
        for vocal in list:
           crops_dict[vocal]=1
    for i,temp in enumerate(crops_dict):
        crops_dict[temp]=i
    return crops_dict
crops_dict=build_dict()
max_len=len(crops_dict)

In [49]:
wordvec_maxLen=256

def build_vector(sentences):
    res=[]
    for sentence in sentences:
        temp=[0 for i in range(wordvec_maxLen)]
        words=sentence.split(' ')
        for i,word in enumerate(words):
            if i<wordvec_maxLen:
                temp[i]=crops_dict[word]
            else:
                break
        res.append(temp)
    return np.array(res)
train_x=build_vector(train_x)
test_x=build_vector(test_x)
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
train_y = encoder.fit_transform(train_y)
test_y=encoder.fit_transform(test_y)

In [51]:

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers.merge import concatenate
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Activation, merge, Input, Lambda, Reshape
from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional
from keras.utils.np_utils import to_categorical
from keras import initializers
from keras import backend as K
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

In [41]:
model = Sequential()

model.add(Embedding(input_dim=max_len,output_dim=64,input_length=256))
#构建LSTM层
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(32,activation='relu'))
model.add(Dense(3, activation='softmax'))
 
#模型可视化
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 256, 64)           12141376  
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16384)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                524320    
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 99        
Total params: 12,665,795
Trainable params: 12,665,795
Non-trainable params: 0
_________________________________________________________________


In [42]:
#激活神经网络 
model.compile(optimizer = 'adam',              #RMSprop优化器
              loss = 'categorical_crossentropy',       #二元交叉熵损失
              metrics = ['acc']                   #计算误差或准确率
              )
 
#训练
history = model.fit(train_x, train_y, batch_size=32, epochs=10, 
                    verbose=True, validation_data=(test_x, test_y))
 
#----------------------------------预测与可视化------------------------------
#预测
score = model.evaluate(test_x,test_y, batch_size=32)
print('test loss:', score[0])
print('test accuracy:', score[1])

Epoch 1/10
241/241 [==============================] - 26s 104ms/step - loss: 0.9674 - acc: 0.5238 - val_loss: 0.8331 - val_acc: 0.6132
Epoch 2/10
241/241 [==============================] - 24s 101ms/step - loss: 0.3343 - acc: 0.8794 - val_loss: 0.7850 - val_acc: 0.6584
Epoch 3/10
241/241 [==============================] - 24s 101ms/step - loss: 0.0705 - acc: 0.9820 - val_loss: 0.8087 - val_acc: 0.6812
Epoch 4/10
241/241 [==============================] - 24s 102ms/step - loss: 0.0270 - acc: 0.9944 - val_loss: 0.8330 - val_acc: 0.6786
Epoch 5/10
241/241 [==============================] - 24s 102ms/step - loss: 0.0169 - acc: 0.9956 - val_loss: 0.8375 - val_acc: 0.6864
Epoch 6/10
241/241 [==============================] - 24s 101ms/step - loss: 0.0107 - acc: 0.9964 - val_loss: 0.8614 - val_acc: 0.6802
Epoch 7/10
241/241 [==============================] - 24s 101ms/step - loss: 0.0087 - acc: 0.9969 - val_loss: 0.8791 - val_acc: 0.6937
Epoch 8/10
241/241 [==============================] - 2

In [43]:
model.save('./model/embedding+NN.h5')

In [52]:
from keras.models import load_model
model=load_model('./model/TD-IDF+NN.h5')
pre_y=np.argmax(model.predict(test_x),axis=1)
test_y=t_y
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
test_accracy = accuracy_score(test_y,pre_y)
F1_score=f1_score(test_y,pre_y,average='macro')
Recall_score=recall_score(test_y,pre_y,average='macro')
Precision_score=precision_score(test_y,pre_y,average='macro')
print("Acc=",test_accracy)
print("Precision_score",Precision_score)
print("Recall_score",Recall_score)
print("F1_score",F1_score)

ValueError: in user code:

    c:\Users\a\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    c:\Users\a\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\Users\a\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\Users\a\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\Users\a\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\Users\a\